# import

In [17]:
import openpyxl
import math

# Загрузка данных из .xlsx

In [14]:
default_path = '3302_Kalugina_Sarzhevskiy.xlsx'

path = input('Введите путь к .xlsx таблице (оставьте пустым для загрузки из таблицы по умолчанию): ')

wb = openpyxl.load_workbook(default_path) if path == '' else openpyxl.load_workbook(path)
sh = wb.active

input_data = []

for i in range(1, 301):
    input_data.append(float(sh.cell(row=i, column=1).value))

if path == '':
    print(f"Загружена трасса из {len(input_data)} из файла {default_path}")
else:
    print(f"Загружена трасса из {len(input_data)} из файла {path}")

Введите путь к .xlsx таблице (оставьте пустым для загрузки из таблицы по умолчанию): 
Загружена трасса из 300 из файла 3302_Kalugina_Sarzhevskiy.xlsx


# Оценки мат. ожидания, дисперсии и ср. кв. отклонения

In [18]:
expected_value = sum(input_data) / len(input_data)
print(f"Оценка математического ожидания: {expected_value}")

dispersion = sum([(x - expected_value)**2 for x in input_data]) / (len(input_data) - 1)
print(f"Оценка диперсии: {dispersion}")

standard_deviation = math.sqrt(dispersion / len(input_data))
print(f"Оценка среднеквадратического отклонения: {standard_deviation}")

Оценка математического ожидания: 1.63875903026897
Оценка диперсии: 11.280630849539067
Оценка среднеквадратического отклонения: 0.19391261648432492
